
# Basic Noise Cancelling Program



In [1]:
import os

if not os.path.exists('./tmp'):
	os.makedirs('./tmp')

In [2]:
from pytube import YouTube

def download_audio_from_youtube(url, filename, dest='./tmp'):
	yt = YouTube(url)
	stream = yt.streams.filter(only_audio=True).first()
	stream.download(output_path=dest, filename=filename)

In [3]:
noise_video_url = 'https://www.youtube.com/watch?v=FeOrG8FrNko'
clean_video_url = 'https://www.youtube.com/watch?v=FJVcGMcEw7Q'

download_audio_from_youtube(noise_video_url, 'noise')
download_audio_from_youtube(clean_video_url, 'clean')

In [4]:
from pydub import AudioSegment

def clamp_audios(audio1_path, audio2_path):
	audio1 = AudioSegment.from_file(audio1_path)
	audio2 = AudioSegment.from_file(audio2_path)
	print('Durations before(s): ', len(audio1), len(audio2))
	shortest_duration = min(len(audio1), len(audio2))
	audio1 = audio1[:shortest_duration]
	audio2 = audio2[:shortest_duration]
	print('Durations after(s): ', len(audio1), len(audio2))
	return audio1, audio2

In [5]:
def normalize_volume(audio1, audio2):
	print("Volumes before(dBFS): ", audio1.dBFS, audio2.dBFS)
	target_dbfs = -20
	gain1 = target_dbfs - audio1.dBFS
	gain2 = target_dbfs - audio2.dBFS
	audio1 = audio1.apply_gain(gain1)
	audio2 = audio2.apply_gain(gain2)
	print("Volumes after(dBFS): ", audio1.dBFS, audio2.dBFS)
	return audio1, audio2

In [6]:
noise_path = './tmp/noise'
clean_path = './tmp/clean'

noise, clean = clamp_audios(noise_path, clean_path)
noise, clean = normalize_volume(noise, clean)

Durations before(s):  60372 2331423
Durations after(s):  60372 60372
Volumes before(dBFS):  -9.82339117825276 -19.88362118722665
Volumes after(dBFS):  -20.00212083742674 -19.99946987154684


In [7]:
combined = noise.overlay(clean)
combined.export('./tmp/combined.wav', format='wav')

<_io.BufferedRandom name='./tmp/combined.wav'>